# Support Vector Regression (SVR) Model for Deconvolution of Human Immune Cell Types
### Non-Negative Least Square Problem


$$ min_A(||AS-X||^2), s.t. \bigg\{ \sum_{i=1}^{N} a_{ki}=1 , a_{ki}\geq0, $$

$$x_{ij}=\sum_{i=1}^{N} a_{ki}s_{ij}$$


x_ij: g ene expression level of gene j in a sample k 

a_ki: i cell type proportion for sample k 

s_ij: gene expression level of gene j in i cell type
 
 More generally, matrix form of the problem is:
$$ X=AS$$
X: mixture data 

A: proportion matrix ( desired matrix )

S: signature matrix 






In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVR
from sklearn.metrics import r2_score


# note that these sample signature and mixture data are already normalized.!
sample_mixture =pd.read_csv('sample_mixture.csv')
sample_mixture=sample_mixture.set_index('Unnamed: 0')
sample_signature=pd.read_csv('sample_signature.csv')
sample_signature=sample_signature.set_index('Unnamed: 0')
sample_signature=sample_signature.T
sample_signature.head()

Unnamed: 0,CD8_naive,CD8_CM,CD8_EM,CD8_TE,MAIT,VD2+,VD2-,TFH,Treg,Th1/Th17,...,C_mono,I_mono,NC_mono,NK,pDC,mDC,Neutrophils,Basophils,CD4_TE,Th1
ENSG00000074800.13,0.019306,0.019509,0.024111,0.014170,0.018945,0.017441,0.018557,0.017653,0.019713,0.020704,...,0.046601,0.042084,0.037145,0.026518,0.023705,0.042568,0.002676,0.004356,0.018607,0.021899
ENSG00000142657.20,0.001534,0.001310,0.001771,0.001175,0.001052,0.001448,0.001559,0.001402,0.001425,0.001377,...,0.016848,0.011849,0.008149,0.002751,0.012342,0.006728,0.012204,0.063487,0.001485,0.001596
ENSG00000028137.18,0.000521,0.002110,0.005592,0.003650,0.001996,0.003436,0.003856,0.002627,0.008584,0.004426,...,0.043709,0.057635,0.057548,0.010829,0.003468,0.006674,0.011901,0.000061,0.007298,0.004487
ENSG00000142676.12,0.029999,0.024920,0.011015,0.011725,0.016921,0.016165,0.012317,0.021317,0.028910,0.027441,...,0.015015,0.021493,0.013401,0.012447,0.017637,0.020058,0.000763,0.006279,0.013762,0.032776
ENSG00000169442.8,0.012729,0.016353,0.017618,0.016800,0.011174,0.016748,0.011938,0.014729,0.022241,0.023453,...,0.011153,0.024896,0.013579,0.009212,0.000306,0.007565,0.002273,0.002739,0.017991,0.025786


In [2]:
signature_coarse=pd.read_csv('signature_coarse.csv')
signature_coarse=signature_coarse.set_index('gene')
signature_coarse.head()

,CD8,CD4,B_cell,NK,Neutrophils,monocyte,Fibroblast,Endothelial
gene,,,,,,,,
ENSG00000000971,0.000000,0.000044,0.000000,0.000045,0.000003,0.000010,0.006356,0.009847
ENSG00000003436,0.000000,0.000000,0.000007,0.000200,0.000095,0.000022,0.003832,0.039887
ENSG00000008517,0.054930,0.023485,0.000031,0.018015,0.000198,0.000057,0.000097,0.014840
ENSG00000008988,0.009017,0.010958,0.166875,0.005670,0.000231,0.057349,0.018695,0.120254
ENSG00000009790,0.010526,0.003494,0.008991,0.021176,0.002923,0.006066,0.000027,0.000087


In [3]:
signature_coarse.shape

(355, 8)

In [4]:
def Nu_SVR(mixture,signature):
    
   #function for having same genes for mixture and signature matrix'
    gene_signature=signature.index.values  
    gene_rna=mixture.index.values
    genes=[genes for genes in gene_rna if genes not in gene_signature] 
    mixture=mixture.drop(index=genes)
    
    
    gene_mixture=mixture.index.values 
    genes_for_signature=[genes for genes in gene_signature if genes not in gene_mixture]    
    signature=signature.drop(index=genes_for_signature)
    
           
    cell_type_data=pd.DataFrame(columns=signature.columns.values.tolist())
    for column in mixture: 
        #support vector regression 
            regr_nusvr = NuSVR(kernel='linear',nu=1)
            train_x=np.asanyarray(signature)
            train_y=np.asanyarray(mixture[column])
            model=regr_nusvr.fit (train_x, train_y)
            result=model.coef_
            #make the negative values zero, and normalized to sum up 1
            result=np.where(result<0, 0, result) 
            normalizer = 1 /  np.sum(result)
            estimated_cell_fraction= [x * normalizer for x in result]
            cell_type=pd.DataFrame(estimated_cell_fraction,index=[column],columns=signature.columns.values.tolist())
            cell_type_data=cell_type_data.append(cell_type)
     
    return cell_type_data

In [5]:
def Linear_SVR(mixture,signature,cell_number):
    'main function to find the proportion matrix'
    "Support Vector Regression "
    
    cell_type_data=pd.DataFrame(columns=signature.columns.values.tolist())
    for column in mixture: 
        #support vector regression 
            regr_svr = LinearSVR(fit_intercept=False,max_iter=2000)
            train_x=np.asanyarray(signature)
            train_y=np.asanyarray(mixture[column])
            model=regr_svr.fit (train_x, train_y)
            result=model.coef_
            result=np.reshape(result,(1,cell_number))
           
            #make the negative values zero, and normalized to sum up 1
            result=np.where(result<0, 0, result) 
            normalizer = 1 /  np.sum(result)
            estimated_cell_fraction= [x * normalizer for x in result]
            cell_type=pd.DataFrame(estimated_cell_fraction,index=[column],columns=signature.columns.values.tolist())
            cell_type_data=cell_type_data.append(cell_type)
    
    return cell_type_data

In [6]:
def generate_test_data_score_NUSVR(samples,cell_number,gene,signature_matrix):
    '''
    
    This function gives the scores of randomly generated synthetic mixture data set for NU_SVR model
       
       sample:  number of samples for synthetic mixture data
       cell_number: # number cell_types
       gene: number of genes in the signature matrix
       mixture data:  threshold mixture data
       signature data : threshold signature matrix'''
    
    #generating synthetic mixture data 
    test_data=np.empty(shape=[samples,cell_number])
    test_mixture=np.empty(shape=[samples,gene])
    test_mixture_df=pd.DataFrame(index= signature_matrix.index)

    result_test=pd.DataFrame(index=range(samples),columns=signature_matrix.columns.values.tolist())
    score_data=np.empty(shape=[samples,1])
    for i in range(samples):
        test_data[i]=np.array(np.random.random(cell_number))
        test_data[i] /=np.sum(test_data[i])
        test_mixture[i]=np.matmul(test_data[i],signature_matrix.T)
        test_mixture_df[i]=pd.DataFrame(test_mixture[i],index= signature_matrix.index)

        test_mixture_df=test_mixture_df.fillna(0)
        #using synthetic mixture data in the model:
        result_test=Nu_SVR(test_mixture_df, signature_matrix)
        score_data[i]=r2_score(test_data[i],result_test.iloc[i])
    return score_data


In [7]:
def generate_test_data_score_LINEAR_SVR(samples,cell_number, gene,signature_matrix):
    '''
    This function gives the scores of randomly generated synthetic mixture data set for LINEAR_SVR model
       
       sample:  number of samples for synthetic mixture data
       cell_number: # number cell_types
       gene: number of genes in the signature matrix
       mixture data:  normalized mixture data
       signature data : normalized signature matrix'''
    
    #generating synthetic mixture data 
    test_data=np.empty(shape=[samples,cell_number])
    test_mixture=np.empty(shape=[samples,gene])
    test_mixture_df=pd.DataFrame(index= signature_matrix.index)
    result_test=pd.DataFrame(index=range(samples),columns=signature_matrix.columns.values.tolist())
    score_data=np.empty(shape=[samples,1])
    for i in range(samples):
        test_data[i]=np.array(np.random.random(cell_number))
        test_data[i] /=np.sum(test_data[i])
        test_mixture[i]=np.matmul(test_data[i],signature_matrix.T)
        test_mixture_df[i]=pd.DataFrame(test_mixture[i],index= signature_matrix.index)
        test_mixture_df=test_mixture_df.fillna(0)
        #using synthetic mixture data in the model:
        result_test=Linear_SVR(test_mixture_df, signature_matrix,cell_number)
        
        score_data[i]=r2_score(test_data[i],result_test.iloc[i])
    return score_data


In [8]:
b=generate_test_data_score_LINEAR_SVR(100,8,355,signature_coarse)
average_score_l=np.mean(b)
average_score_l

0.9999999999999956

In [9]:
a=generate_test_data_score_NUSVR(100,8,355,signature_coarse)
average_score=np.mean(a)
average_score

0.9998813439572684